<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# OpenAI


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

## 基本用法


#### 使用提示调用`complete`


In [ ]:
from llama_index.llms.openai import OpenAI

resp = OpenAI().complete("Paul Graham is ")

In [ ]:
print(resp)

a computer scientist, entrepreneur, and venture capitalist. He is best known for co-founding the startup accelerator Y Combinator and for his influential essays on startups and technology. Graham has also founded several successful companies, including Viaweb (which was acquired by Yahoo) and the social news website Reddit. He is considered a thought leader in the tech industry and has been a vocal advocate for startup culture and innovation.


#### 使用消息列表调用 `chat`


In [ ]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = OpenAI().chat(messages)

In [ ]:
print(resp)

assistant: Ahoy matey! The name's Captain Rainbowbeard, the most colorful pirate on the seven seas! What can I do for ye today? Arrr!


## 流式处理


使用 `stream_complete` 终端点


In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI()
resp = llm.stream_complete("Paul Graham is ")

In [ ]:
for r in resp:
    print(r.delta, end="")

a computer scientist, entrepreneur, and venture capitalist. He is best known for co-founding the startup accelerator Y Combinator and for his work on programming languages and web development. Graham is also a prolific writer and has published several influential essays on technology, startups, and entrepreneurship.

使用 `stream_chat` 端点


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage

llm = OpenAI()
messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.stream_chat(messages)

In [ ]:
for r in resp:
    print(r.delta, end="")

Ahoy matey! The name's Captain Rainbowbeard! Aye, I be a pirate with a love for all things colorful and bright. Me beard be as vibrant as a rainbow, and me ship be the most colorful vessel on the seven seas! What can I do for ye today, me hearty?

## 配置模型


In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
resp = llm.complete("Paul Graham is ")

In [ ]:
print(resp)



Paul Graham is an entrepreneur, venture capitalist, and computer scientist. He is best known for his work in the startup world, having co-founded the accelerator Y Combinator and investing in hundreds of startups. He is also a prolific writer, having authored several books on topics such as startups, programming, and technology.


In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

In [ ]:
print(resp)

assistant: 
My name is Captain Jack Sparrow.


## 函数调用

OpenAI模型原生支持函数调用。这与LlamaIndex工具的抽象相方便地集成在一起，让您可以将任意的Python函数插入到LLM中。

在下面的示例中，我们定义了一个生成Song对象的函数。


In [ ]:
from pydantic import BaseModel
from llama_index.llms.openai.utils import to_openai_tool
from llama_index.core.tools import FunctionTool


class Song(BaseModel):
    """带有名称和艺术家的歌曲"""

    name: str
    artist: str


def generate_song(name: str, artist: str) -> Song:
    """使用提供的名称和艺术家生成一首歌曲。"""
    return Song(name=name, artist=artist)


tool = FunctionTool.from_defaults(fn=generate_song)

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call([tool], "Generate a song")
print(str(response))

name='Sunshine' artist='John Smith'


我们也可以进行多个函数的调用。


In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [tool],
    "Generate five songs from the Beatles",
    allow_parallel_tool_calls=True,
)
for s in response.sources:
    print(f"Name: {s.tool_name}, Input: {s.raw_input}, Output: {str(s)}")

Name: generate_song, Input: {'args': (), 'kwargs': {'name': 'Hey Jude', 'artist': 'The Beatles'}}, Output: name='Hey Jude' artist='The Beatles'
Name: generate_song, Input: {'args': (), 'kwargs': {'name': 'Let It Be', 'artist': 'The Beatles'}}, Output: name='Let It Be' artist='The Beatles'
Name: generate_song, Input: {'args': (), 'kwargs': {'name': 'Yesterday', 'artist': 'The Beatles'}}, Output: name='Yesterday' artist='The Beatles'
Name: generate_song, Input: {'args': (), 'kwargs': {'name': 'Come Together', 'artist': 'The Beatles'}}, Output: name='Come Together' artist='The Beatles'
Name: generate_song, Input: {'args': (), 'kwargs': {'name': 'Help!', 'artist': 'The Beatles'}}, Output: name='Help!' artist='The Beatles'


## 结构化预测

函数调用的一个重要用例是提取结构化对象。LlamaIndex通过`structured_predict`提供了一个直观的接口，只需定义目标Pydantic类（可以是嵌套的），然后给定一个提示，我们就可以提取出所需的对象。


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate
from pydantic import BaseModel


class Restaurant(BaseModel):
    """具有名称、城市和美食的餐厅。"""

    name: str
    city: str
    cuisine: str


llm = OpenAI(model="gpt-3.5-turbo")
prompt_tmpl = PromptTemplate(
    "在给定的城市{city_name}生成一家餐厅"
)
restaurant_obj = llm.structured_predict(
    Restaurant, prompt_tmpl, city_name="旧金山"
)

In [ ]:
restaurant_obj

Restaurant(name='Tasty Bites', city='San Francisco', cuisine='Italian')

## 异步


In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
resp = await llm.acomplete("Paul Graham is ")

In [ ]:
print(resp)

a computer scientist, entrepreneur, and venture capitalist. He is best known for co-founding the startup accelerator Y Combinator and for his work as an essayist and author on topics related to technology, startups, and entrepreneurship. Graham is also the co-founder of Viaweb, one of the first web-based applications, which was acquired by Yahoo in 1998. He has been a prominent figure in the tech industry for many years and is known for his insightful and thought-provoking writings on a wide range of subjects.


In [ ]:
resp = await llm.astream_complete("Paul Graham is ")

In [ ]:
async for delta in resp:
    print(delta.delta, end="")



Paul Graham is an entrepreneur, venture capitalist, and computer scientist. He is best known for his work in the startup world, having co-founded the accelerator Y Combinator and investing in many successful startups such as Airbnb, Dropbox, and Stripe. He is also a prolific writer, having authored several books on topics such as startups, programming, and technology.

异步函数调用也是支持的。


In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")
response = await llm.apredict_and_call([tool], "Generate a song")
print(str(response))

name='Sunshine' artist='John Smith'


## 在每个实例级别设置API密钥
如果需要的话，您可以让不同的LLM实例使用不同的API密钥。


In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", api_key="BAD_KEY")
resp = OpenAI().complete("Paul Graham is ")
print(resp)

a computer scientist, entrepreneur, and venture capitalist. He is best known as the co-founder of the startup accelerator Y Combinator. Graham has also written several influential essays on startups and entrepreneurship, which have gained a wide following in the tech industry. He has been involved in the founding and funding of numerous successful startups, including Reddit, Dropbox, and Airbnb. Graham is known for his insightful and often controversial opinions on various topics, including education, inequality, and the future of technology.
